In [1]:
import smtplib

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

import os
import getpass


import win32com.client

In [2]:
def email_sender(email_from, email_to, email_subject, email_body, attachment_pathlist=[]):
    msg = MIMEMultipart()
    msg['FROM'] = email_from
    msg['To'] = email_to
    msg['Subject'] = email_subject

    body = email_body
    msg.attach(MIMEText(body, 'plain'))

    if len(attachment_pathlist)>0:
        for file in attachment_pathlist:
            name = os.path.basename(file)
            part = MIMEBase('application', 'octet-stream')

            with open(file, 'rb') as file:
                part.set_payload(file.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', "attachment; filename= %s" % name)

            msg.attach(part)

    text = msg.as_string()
    
    server = smtplib.SMTP('srvmail')
    server.sendmail(email_from, email_to, text)
    server.quit()

In [3]:
def email_confirmation(cliente, search_string, data, folders_search = [], main_folder='Inbox'):
    path=r'C:\Users\%s\OneDrive - Banco BTG Pactual S.A\Documents\offshore' % getpass.getuser()
    outlook = win32com.client.Dispatch('outlook.application')
    namespace = outlook.GetNamespace("MAPI")
    root_folder = namespace.Folders.Item(2)
    
    cliente = cliente.lower()
    data = data[-4:]+data[3:5]+data[:2]
    
    if os.path.isdir(path+'\\'+cliente)==True:
        os.mkdir(path+'\\'+cliente+'\\'+data)
    else:
        os.mkdir(path+'\\'+cliente)
        os.mkdir(path+'\\'+cliente+'\\'+data)
    n_path = path+'\\'+cliente+'\\'+data
    
    for folder in folders_search:
        subfolder = root_folder.Folders['Caixa de Entrada'].Folders['confirmations']
        message = subfolder.Items
    
        for msg in message:
            body_content = msg.body
            msg_sub = msg.Subject
            if (search_string in body_content)&(data in msg_sub):
                count_attachments = msg.Attachments.Count
        
                string = msg_sub.split(" ")
                for i in range(0, len(string)):
                    if string[i] == 'of':
                        ticker = string[i+1]
                        if '*' in ticker:
                            ticker = ticker.replace("*", '')
                        elif "&" in ticker:
                            ticker = ticker.replace("&", '')

                if count_attachments > 0:
                    for item in range(count_attachments):
                        file_name = msg.Attachments.Item(item + 1).Filename
                        if '.pdf' in file_name:
                            attach = msg.Attachments.Item(item+1)
                            attach.SaveAsFile(n_path+'\\'+ticker+'.pdf')

In [5]:
email_confirmation('Claritas', 'CLARITAS', '18/05/2018', folders_search =['confirmations'], main_folder='Caixa de Entrada')

In [ ]:
email_sender('guilherme.correia@btgpactual.com', 'guilherme.correia@btgpactual.com', 
              'teste_class', 'this is a test <br> Qualquer dúvida, estamos à disposição', 
              [r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\calypso_intc.xlsx',
              r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\calypso_asset.xlsx'])